Competitions included in this report:


In [1]:
# Enable PEP8 code checking.
%load_ext pycodestyle_magic

%pycodestyle_on

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import time
import yaml

from football_data_api import data_fetchers
from footy import Footy
from footy.domain.Fixture import Fixture
from footy.domain.Team import Team

with open('footy.yml') as stream:
    footy_data = yaml.safe_load(stream.read())


def extract_team_names_from_matches(matches):
    team_names = []

    for match in matches:
        away_team_name = match['awayTeam']['name']
        home_team_name = match['homeTeam']['name']

        for team_name in [away_team_name, home_team_name]:
            if team_name not in team_names:
                team_names.append(team_name)

    return sorted(team_names)


def moving_average(x, w=3):
    a = (np.convolve(x, np.ones(w), 'valid') / w).tolist()
    return x.tolist()[0:w - 1] + a


def process_competition_page(competition_code):
    time.sleep(10)  # Throttle calls to the Football Data API
    teams = []
    football_data_api = data_fetchers.CompetitionData()
    widget = footy_data['competitions'][competition_code]
    competition_name = widget['competition_name']
    football_data_api.competition = competition_name
    matches_data = football_data_api.get_info('matches')
    team_names = extract_team_names_from_matches(matches_data['matches'])
    match = matches_data['matches'][0]
    current_match_day = match['season']['currentMatchday']
    footy = Footy()
    total_goals_scored_by_home_team = 0
    total_goals_scored_by_away_team = 0
    total_games = 0

    for team_name in team_names:
        team = Team(team_name)
        footy.add_team(team)

    for match_day in range(0, current_match_day):
        fixture_params = []

        for match in matches_data['matches']:
            match_status = match['status']

            if match['matchday'] != match_day:
                continue
            elif match_status != 'FINISHED':
                continue
            elif match['stage'] != 'REGULAR_SEASON':
                continue

            total_games += 1
            home_team_name = match['homeTeam']['name']
            home_team = footy.get_team(home_team_name)
            home_team_goals = match['score']['fullTime']['homeTeam']
            away_team_goals = match['score']['fullTime']['awayTeam']
            goals_for = match['score']['fullTime']['homeTeam']
            total_goals_scored_by_home_team += goals_for
            goals_for += home_team.goals_for()
            goals_against = match['score']['fullTime']['awayTeam']
            total_goals_scored_by_away_team += goals_against
            goals_against += home_team.goals_against()
            home_games = home_team.home_games() + 1
            points = home_team.points()

            if home_team_goals > away_team_goals:
                points += 3
                outcome = [1, 0, 0]
            elif home_team_goals == away_team_goals:
                points += 1
                outcome = [0, 1, 0]
            else:
                outcome = [0, 0, 1]

            home_team.goals_for(goals_for)
            home_team.goals_against(goals_against)
            home_team.home_games(home_games)
            home_team.points(points)

            away_team_name = match['awayTeam']['name']
            away_team = footy.get_team(away_team_name)
            fixture_params.append((home_team_name, away_team_name))
            goals_for = away_team_goals
            goals_for += away_team.goals_for()
            goals_against = match['score']['fullTime']['awayTeam']
            goals_against += away_team.goals_against()
            away_games = away_team.away_games() + 1
            points = away_team.points()

            if home_team_goals < away_team_goals:
                points += 3
            elif home_team_goals == away_team_goals:
                points += 1

            away_team.goals_for(goals_for)
            away_team.goals_against(goals_against)
            away_team.away_games(away_games)
            away_team.points(points)

            footy.add_team(home_team)
            footy.add_team(away_team)
            total_goals_scored_by_home_team += home_team_goals
            total_goals_scored_by_away_team += away_team_goals

        if total_games > 0:
            average_goals_scored_by_a_home_team = \
                total_goals_scored_by_home_team
            average_goals_scored_by_a_home_team /= total_games
            footy.average_goals_scored_by_a_home_team(
                average_goals_scored_by_a_home_team)

            average_goals_scored_by_an_away_team = \
                total_goals_scored_by_away_team
            average_goals_scored_by_an_away_team /= total_games
            footy.average_goals_scored_by_an_away_team(
                average_goals_scored_by_an_away_team)

        for team_name in footy.get_team_names():
            team = footy.get_team(team_name)
            attack_strength = footy.attack_strength(team)
            defence_factor = footy.defence_factor(team)
            team.historic_attack_strength(attack_strength)
            team.historic_defence_factor(defence_factor)
            footy.add_team(team)

        for fixture_param in fixture_params:
            (home_team_name, away_team_name) = fixture_param
            home_team = footy.get_team(home_team_name)
            away_team = footy.get_team(away_team_name)
            fixture = footy.fixture(home_team, away_team)

            y_prob = fixture.outcome_probabilities()

            if y_prob is not None:
                print(f'DEBUG: y_prob {y_prob}')
                brier_score = footy.brier_score(outcome, y_prob)
                home_team.historic_briers_score(brier_score)
                footy.add_team(home_team)
                away_team.historic_briers_score(brier_score)
                footy.add_team(away_team)

    return footy


def show_league_table(footy):
    df = footy.dataframe().rename(columns={
        'attack_strength': 'AS',
        'away_games': 'AG',
        'defence_factor': 'DF',
        'goal_difference': 'GD',
        'goals_against': 'GA',
        'goals_for': 'GF',
        'home_games': 'HG',
        'points': 'PTS'
    })
    print(df)
    print('')
    print('Keys:')
    print(' - AS = Attack Strength')
    print(' - AG = Away Games Played')
    print(' - DF = Defence Factor')
    print(' - GA = Goals Against')
    print(' - GD = Goal Difference')
    print(' - GF = Goals For')
    print(' - HG = Home Games')
    print(' - PTS = Points')


def show_team(team):
    team_name = team.team_name()
    outcomes_briers_scores = team.historic_briers_score()

    if len(outcomes_briers_scores) >= 3:
        briers_scores_outcomes_ma3 = moving_average(outcomes_briers_scores)
    else:
        briers_scores_outcomes_ma3 = None

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(team_name)

    if len(outcomes_briers_scores) > 0:
        label = f'Briers Score: {outcomes_briers_scores[-1]}'
    else:
        label = 'Briers Score'

    ax1.plot(outcomes_briers_scores, label=label)
    ax1.legend()
    ax1.set_ylim(0.0, 2.0)
    ax1.set(xlabel='Games', title='Briers Scores')

    attack_strengths = team.historic_attack_strength()
    defence_factors = team.historic_defence_factor()
    ax2.plot(attack_strengths, label='Attack Strengths')
    ax2.plot(defence_factors, label='Defence Factors')
    ax2.legend()
    ax2.set(xlabel='Games', title='Attack Strengh/Defence Factor')
    ax2.grid()


def show_teams(footy):
    for team_name in footy.get_team_names():
        team = footy.get_team(team_name)
        show_team(team)


for competition_code in footy_data['competitions']:
    widget = footy_data['competitions'][competition_code]
    competition_name = widget['competition_name']
    bbc_table = widget['bbc_table_site']
    country = widget['country']
    print(f'- {competition_name} ({country})')

140:1: W293 blank line contains whitespace


- Bundesliga (DE)
- Série A (BR)
- Eredivisie (NE)
- Championship (UK)
- Ligue 1 (FR)
- Primera Division (ES)
- Premier League (UK)
- Primeira Liga (PT)
- Serie A (IT)


![http://creativecommons.org/licenses/by-sa/4.0/](images/cc-by-sa-80x15.png)

This work is licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-sa/4.0/).

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak 
</div>

# German Bundesliga

## League Table

In [3]:
footy = process_competition_page('BL1')
show_league_table(footy)

DEBUG: y_prob [6.67, 5.13, 9.48]


ValueError: y_prob contains values greater than 1.

## Teams

In [ ]:
show_teams(footy)

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak 
</div>

# Brazilian Série A

In [ ]:
# footy = process_competition_page('BSA')
# show_league_table(footy)

## Teams

In [ ]:
# show_teams(footy)

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak 
</div>

# Dutch Eredivisie

In [ ]:
# competition = process_competition_page1('DED')

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak 
</div>

# English Championship

In [ ]:
# competition = process_competition_page1('ELC')

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak 
</div>

# French Ligue 1

In [ ]:
# competition = process_competition_page1('FL1')

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak 
</div>

# Spanish Primera Division

In [ ]:
# competition = process_competition_page1('PD')

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak 
</div>

# English Premier League

In [ ]:
# competition = process_competition_page1('PL')

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak 
</div>

# Portuguese  Primeira Liga

In [ ]:
# competition = process_competition_page1('PPL')

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak 
</div>

# Italian Serie A

In [ ]:
# competition = process_competition_page1('SA')